- 7B, 13B, 70B의 의미
1. 7B: 7 billion(70억) 개의 매개변수를 가진 모델
2. 13B: 13 billion(130억) 개의 매개변수를 가진 모델
3. 70B: 70 billion(700억) 개의 매개변수를 가진 모델

- 라마 모델에서 7B, 13B, 70B는 각각 모델의 크기를 나타낸다. 이 값은 모델이 가지는 매개변수(parameter)의 수를 나타내며, 일반적으로 모델의 크기가 클수록 더 많은 매개변수를 가지고 있다.

- 일반적으로 모델의 크기가 크면 더 많은 매개변수를 가지므로 더 많은 문맥을 이해하고 더 복잡한 패턴을 학습할 수 있다. 따라서 큰 모델을 사용하면 성능이 더 우수하지만, 더 많은 계산 리소스와 메모리를 필요로 하기 때문에 학습과 추론에 더 많은 시간이 소요된다.

Llama poc(cpu)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from transformers import AutoTokenizer
import transformers
import torch

# 모델로딩
model = "meta-llama/Llama-2-7b-chat-hf" #Llama-2-7b-chat model load
tokenizer = AutoTokenizer.from_pretrained(
    model,
    use_auth_token= MY_AUTH_TOKEN,
)

# 파이프라인 설정
pipeline = transformers.pipeline(
    "text-generation",  #반환 정의
    model=model,
    torch_dtype=torch.float16,
    device_map="auto", #cpu, gpu
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:31<00:00, 15.82s/it]
Some parameters are on the meta device because they were offloaded to the disk.
Device set to use mps


In [ ]:
from translate import Translator

# 텍스트 생성 함수
def generate(x, max_length=100):
    sequences = pipeline(
        x,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=max_length,
    )

    return sequences[0]["generated_text"].replace(x, "")

translator_ko_en = Translator(from_lang='ko', to_lang='en')
translator_en_ko = Translator(from_lang='en', to_lang='ko')

In [5]:
input_ko = "유진이의 마음을 사로잡으려면 어떻게 해야할까??"
input_en = translator_ko_en.translate(input_ko) 

output_en = generate(input_en,100)
output_ko =  translator_en_ko.translate(output_en)
print(output_en)
print(output_ko)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/Users/keemsub/Desktop/stockify/stockify/lib/stockify/lib/python3.9/site-packages/transformers/pytorch_utils.py:333: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)


RuntimeError: MPS backend out of memory (MPS allocated: 6.66 GB, other allocations: 12.31 MB, max allowed: 6.80 GB). Tried to allocate 250.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).